-- =========================
-- ===== SILVER LAYER =====
-- =========================

In [0]:
%sql
-- Set default schema for 2-part names
USE SCHEMA sales_sv;

Customers: typing, trimming, normalize empty email to NULL, deduplicate by customer_id (keep latest signup_date)

In [0]:
%sql
CREATE OR REPLACE TABLE sales_sv.customers_sv AS
SELECT
  CAST(customer_id AS BIGINT)                    AS customer_id,
  TRIM(first_name)                               AS first_name,
  TRIM(last_name)                                AS last_name,
  LOWER(NULLIF(email, ''))                       AS email,
  CAST(signup_date AS DATE)                      AS signup_date
FROM sales_bz.customers_bz
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY signup_date DESC) = 1;

Orders: typing, uppercase status, drop NULL amounts, deduplicate by order_id (keep latest order_ts)

In [0]:
%sql
CREATE OR REPLACE TABLE sales_sv.orders_sv AS
SELECT
  CAST(order_id AS BIGINT)                       AS order_id,
  CAST(customer_id AS BIGINT)                    AS customer_id,
  CAST(order_date AS TIMESTAMP)                  AS order_ts,
  UPPER(status)                                  AS status,
  CAST(amount AS DECIMAL(18,2))                  AS amount
FROM sales_bz.orders_bz
WHERE amount IS NOT NULL
QUALIFY ROW_NUMBER() OVER (PARTITION BY order_id ORDER BY order_ts DESC) = 1;

-- Categories: basic typing/trim

In [0]:
%sql
CREATE OR REPLACE TABLE sales_sv.categories_sv AS
SELECT
  CAST(category_id AS BIGINT)                    AS category_id,
  TRIM(category_name)                            AS category_name,
  CAST(parent_category_id AS BIGINT)             AS parent_category_id
FROM common_bz.categories_bronze;